In [1]:
from functools import partial
from pathlib import Path
from typing import Optional, Tuple
import cv2
import fire
import numpy as np
import torch
import torch.nn.functional as F
from accelerate import Accelerator
from PIL import Image
from scipy.sparse.linalg import eigsh
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.decomposition import PCA
from torchvision.utils import draw_bounding_boxes
from tqdm import tqdm
import extract_utils as utils

## Extract Features

In [2]:
# images_list="/home/phdcs2/Hard_Disk/Datasets/Deep-Spectral-Segmentation/data/VOC2012/lists/images.txt"
# images_root="/home/phdcs2/Hard_Disk/Datasets/Deep-Spectral-Segmentation/data/VOC2012/images/trainval/JPEGImages"
# model_name="dino_vits16"
# batch_size=1
# output_dir="/home/phdcs2/Hard_Disk/Datasets/Deep-Spectral-Segmentation/data/VOC2012/features/dino_vits16"
# which_block=-1
# # Output
# utils.make_output_dir(output_dir)
# # Models
# model_name = model_name.lower()
# model, val_transform, patch_size, num_heads = utils.get_model(model_name)    #patch size= 16 number of heads= 6
# print("patch size=", patch_size, "number of heads=", num_heads)
# # Add hook
# if 'dino' in model_name or 'mocov3' in model_name:
#     feat_out = {}
#     def hook_fn_forward_qkv(module, input, output):
#         print("feat_out.keys()", feat_out.keys())
#         feat_out["qkv"] = output
#     model._modules["blocks"][which_block]._modules["attn"]._modules["qkv"].register_forward_hook(hook_fn_forward_qkv)
# else:
#     raise ValueError(model_name)
# # Dataset
# filenames = Path(images_list).read_text().splitlines()
# dataset = utils.ImagesDataset(filenames=filenames, images_root=images_root, transform=val_transform)
# dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, num_workers=8)
# print(f'Dataset size: {len(dataset)=}')
# print(f'Dataloader size: {len(dataloader)=}')
# # Prepare
# accelerator = Accelerator(fp16=True, cpu=False)
# # model, dataloader = accelerator.prepare(model, dataloader)
# model = model.to(accelerator.device)
# model.num_features
# print("model.num_features=", model.num_features)
# model.get_submodule
# # Process
# pbar = list(tqdm(dataloader, desc='Processing'))
# # Process
# # pbar = list(tqdm(dataloader, desc='Processing'))
# print("feat_out.keys()=", feat_out.keys())
# for i, (images, files, indices) in enumerate(pbar[:2]):
#     output_dict = {}
#     print("images.shape=", images.shape, "files =", files, "indices", indices)

#     # Check if file already exists
#     id = Path(files[0]).stem
#     print("id=", id)
#     output_file = Path(output_dir) / f'{id}.pth'
#     # if output_file.is_file():
#     #     pbar.write(f'Skipping existing file {str(output_file)}')
#     #     continue

#     # Reshape image
#     P = patch_size
#     B, C, H, W = images.shape
#     H_patch, W_patch = H // P, W // P
#     H_pad, W_pad = H_patch * P, W_patch * P
#     T = H_patch * W_patch + 1  # number of tokens, add 1 for [CLS]
#     # images = F.interpolate(images, size=(H_pad, W_pad), mode='bilinear')  # resize image
#     images = images[:, :, :H_pad, :W_pad]
#     images = images.to(accelerator.device)
#     print("images.shape after padding=", images.shape)

#     # Forward and collect features into output dict
#     if 'dino' in model_name or 'mocov3' in model_name:
#         # accelerator.unwrap_model(model).get_intermediate_layers(images)[0].squeeze(0)
#         model.get_intermediate_layers(images)[0].squeeze(0)
#         print(model.get_intermediate_layers(images)[0])
#         # output_dict['out'] = out
#         print("feat_out.keys()=", feat_out.keys())
#         output_qkv = feat_out["qkv"].reshape(B, T, 3, num_heads, -1 // num_heads).permute(2, 0, 3, 1, 4)
#         print("type(output_qkv)", type(output_qkv))
#         print("output_qkv.shape", output_qkv.shape)    #3, 1, 6, 931, 64]
#         print("output_qkv[0].shape", output_qkv[0].shape)
#         print("output_qkv[1].shape", output_qkv[1].shape)
#         print("output_qkv[2].shape", output_qkv[2].shape)
#         # output_dict['q'] = output_qkv[0].transpose(1, 2).reshape(B, T, -1)[:, 1:, :]
#         output_dict['k'] = output_qkv[1].transpose(1, 2).reshape(B, T, -1)[:, 1:, :]
#         print("output_dict[k].shape=", output_dict['k'].shape)
#         # output_dict['v'] = output_qkv[2].transpose(1, 2).reshape(B, T, -1)[:, 1:, :]
#     else:
#         raise ValueError(model_name)

#     # print("output_dict.items=", output_dict.items())
#     # Metadata
#     output_dict['indices'] = indices[0]
#     output_dict['file'] = files[0]
#     output_dict['id'] = id
#     output_dict['model_name'] = model_name
#     output_dict['patch_size'] = patch_size
#     output_dict['shape'] = (B, C, H, W)
#     output_dict = {k: (v.detach().cpu() if torch.is_tensor(v) else v) for k, v in output_dict.items()}
#     # for k, v in output_dict.items():
#     #     print("k=", k)
#     #     if torch.is_tensor(v):
#     #         print("v.shape", v.shape)
#     #     else:
#     #         print("v=", v)
#     print("output_dict.keys()", output_dict.keys())
#     print("output_dict['k'].shape=", output_dict['k'].shape,"output_dict['indices'] =", indices[0],"output_dict['file'] =", files[0],"output_dict['id']=" , id, "output_dict['model_name'] =", model_name," output_dict['shape'] =(", B, C, H, W,")output_dict['patch_size'] =",  patch_size )
#     # Save
#     accelerator.save(output_dict, str(output_file))
#     accelerator.wait_for_everyone()
# print(f'Saved features to {output_dir}')

## Extract Eigen Vectors

In [3]:
images_root="/home/phdcs2/Hard_Disk/Datasets/Deep-Spectral-Segmentation/data//object-segmentation/ECSSD/images"
features_dir="/home/phdcs2/Hard_Disk/Datasets/Deep-Spectral-Segmentation/data/object-segmentation/ECSSD/features/dino_vits16"
output_dir="/home/phdcs2/Hard_Disk/Datasets/Deep-Spectral-Segmentation/data/object-segmentation/ECSSD/eigs_mmd_polynomial"
kernel_type='polynomial'
sigma=1.0
degree=3
gamma=None
which_matrix= 'laplacian'
which_color_matrix= 'knn'
which_features= 'k'
normalize=True
threshold_at_zero=True
lapnorm= True
K= 5
image_downsample_factor = None
image_color_lambda = 0.0
multiprocessing = 0

## MMD affinity

In [4]:
# def mmd_affinity_matrix(X, Y=None, kernel_type='gaussian', sigma=1.0, degree=3, gamma=None):
#     """
#     Computes the MMD affinity matrix between two sets of points using the specified kernel function.
#     Args:
#         X (torch.Tensor): A tensor of shape (m, d) representing the first set of points.
#         Y (torch.Tensor): A tensor of shape (n, d) representing the second set of points. If None, Y=X.
#         kernel (str): The kernel function to use. One of 'gaussian', 'linear', or 'polynomial'.
#         sigma (float): The bandwidth parameter for the Gaussian kernel.
#         degree (int): The degree parameter for the polynomial kernel.
#         gamma (float): The scaling parameter for the polynomial kernel.
#     Returns:
#         A tensor of shape (m, n) representing the MMD affinity matrix.
#     """
#     print("X.device", X.device)
#     print("Y.device", Y.device)
#
#     if Y is None:
#         Y = X
#
#     if kernel_type == 'gaussian':
#         Kxx = torch.exp(-torch.square(torch.cdist(X, X)) / (2 * sigma**2))
#         Kyy = torch.exp(-torch.square(torch.cdist(Y, Y)) / (2 * sigma**2))
#         Kxy = torch.exp(-torch.square(torch.cdist(X, Y)) / (2 * sigma**2))
#     elif kernel_type == 'linear':
#         Kxx = torch.matmul(X,X.T)
#         Kyy = torch.matmul(Y,Y.T)
#         Kxy = torch.matmul(X,Y.T)
#     elif kernel_type == 'polynomial':
#         if gamma is None:
#             gamma = 1 / X.shape[1]
#         Kxx = (torch.matmul(X,X.T) + gamma)**degree
#         Kyy = (torch.matmul(Y,Y.T) + gamma)**degree
#         Kxy = (torch.matmul(X,Y.T) + gamma)**degree
#         print("Kxx.device=", Kxx.device)
#         print("Kyy.device=", Kyy.device)
#         print("Kxy.device=", Kxy.device)
#     else:
#         raise ValueError(f'Invalid kernel: {kernel_type}')
#
#     m = X.shape[0]
#     n = Y.shape[0]
#
#     H = torch.eye(m) - (1 / m) * torch.ones((m, m))
#     Kxx_H = torch.matmul(H,torch.matmul(Kxx,H))
#     # Kxx_H = torch.matmul(H,Kxx,H)
#     H = torch.eye(n) - (1 / n) * torch.ones((n, n))
#     Kyy_H = torch.matmul(H,torch.matmul(Kyy,H))
#     # Kyy_H = torch.matmul(H,Kyy,H)
#
#     K = (1 / (m**2)) * torch.trace(Kxx_H) + (1 / (n**2)) * torch.trace(Kyy_H) - (2 / (m * n)) * torch.trace(Kxy @ H)
#
#     return K

In [5]:
def polynomial_kernel(x, y, c=1, d=3):
    # Computes the polynomial kernel between x and y
    return (torch.dot(x, y) + c) ** d

In [ ]:
def rbf_kernel(x, y, sigma=1.0):
    # Computes the RBF kernel between x and y
    diff = x - y
    return torch.exp(-torch.dot(diff, diff) / (2.0 * sigma**2))

In [6]:
# def polynomial_kernel(x, y, degree=2):
#     return (torch.dot(x, y) + 1) ** degree

In [7]:
# Pytorch version
def mmd_affinity_matrix(x, y, kernel_type='gaussian', sigma=1.0):
    """
    Calculate Maximum Mean Discrepancy (MMD) as an affinity matrix between two sets of samples x and y
    """
    # print("x.shape=", x.shape, "y.shape", y.shape)
    n,m=x.shape[0], y.shape[0]
    x_mean = torch.mean(x, dim=0)
    # print("x_mean shape", x_mean.shape)
    y_mean = torch.mean(y, dim=0)
    # print("y_shape", y_mean.shape)
    if kernel_type == 'gaussian':
        # use Gaussian kernel
        x_kernel = torch.exp(-torch.sum((x - x_mean)**2, dim=1) / (2 * sigma**2))
        y_kernel = torch.exp(-torch.sum((y - y_mean)**2, dim=1) / (2 * sigma**2))
    elif kernel_type == 'linear':
        # use linear kernel
        x_kernel = torch.matmul(x, x_mean)
        y_kernel = torch.matmul(y, y_mean)
    elif kernel_type == 'polynomial':
        c=1
        d=3
        x_kernel = torch.zeros((n, n))
        y_kernel = torch.zeros((m, m))
        xy_kernel = torch.zeros((n, m))
        # Compute the kernel matrices
        for i in range(n):
            for j in range(i, n):
                x_kernel[i, j] = x_kernel[j, i] = polynomial_kernel(x[i], x[j], c, d)
        for i in range(m):
            for j in range(i, m):
                y_kernel[i, j] = y_kernel[j, i] = polynomial_kernel(y[i], y[j], c, d)
        for i in range(n):
            for j in range(m):
                xy_kernel[i, j] = polynomial_kernel(x[i], y[j], c, d)
     # Compute the mean of the kernel matrices
        mu_xx = torch.mean(x_kernel)
        mu_yy = torch.mean(y_kernel)
        mu_xy = torch.mean(xy_kernel)
        # Compute the MMD affinity matrix
        mmd = torch.zeros((n+m, n+m))
        for i in range(n):
            for j in range(m):
                mmd[i, j+n] = mmd[j+n, i] = xy_kernel[i, j] - mu_xx/n - mu_yy/m + mu_xy
    else:
        raise ValueError("Unsupported kernel type.")
    if kernel_type!='polynomial':
        mmd = torch.zeros((x.shape[0], y.shape[0]))
        for i in range(x.shape[0]):
            for j in range(y.shape[0]):
                mmd[i, j] = x_kernel[i] + y_kernel[j] - 2 * torch.matmul(x[i], y[j])
    return mmd

In [8]:
# # Pytorch version
# def mmd_affinity_matrix(x, y, kernel_type='gaussian', sigma=1.0):
#     """
#     Calculate Maximum Mean Discrepancy (MMD) as an affinity matrix between two sets of samples x and y
#     """
#     # print("x.shape=", x.shape, "y.shape", y.shape)
#     n,m=x.shape[0], y.shape[0]
#     mmd = torch.zeros((n+m, n+m))
#     x_mean = torch.mean(x, dim=0)
#     # print("x_mean shape", x_mean.shape)
#     y_mean = torch.mean(y, dim=0)
#     # print("y_shape", y_mean.shape)
#     if kernel_type == 'gaussian':
#         # use Gaussian kernel
#         x_kernel = torch.exp(-torch.sum((x - x_mean)**2, dim=1) / (2 * sigma**2))
#         y_kernel = torch.exp(-torch.sum((y - y_mean)**2, dim=1) / (2 * sigma**2))
#     elif kernel_type == 'linear':
#         # use linear kernel
#         x_kernel = torch.matmul(x, x_mean)
#         y_kernel = torch.matmul(y, y_mean)
#     elif kernel_type == 'polynomial':
#         c=1
#         d=3
#         x_kernel = torch.zeros((n, n))
#         y_kernel = torch.zeros((m, m))
#         xy_kernel = torch.zeros((n, m))
#         # Compute the kernel matrices
#         for i in range(n):
#             for j in range(i, n):
#                 x_kernel[i, j] = x_kernel[j, i] = polynomial_kernel(x[i], x[j], c, d)
#         for i in range(m):
#             for j in range(i, m):
#                 y_kernel[i, j] = y_kernel[j, i] = polynomial_kernel(y[i], y[j], c, d)
#         for i in range(n):
#             for j in range(m):
#                 xy_kernel[i, j] = polynomial_kernel(x[i], y[j], c, d)
#          # Compute the mean of the kernel matrices
#         mu_xx = torch.mean(x_kernel)
#         mu_yy = torch.mean(y_kernel)
#         mu_xy = torch.mean(xy_kernel)
#         # Compute the MMD affinity matrix
        # Compute the MMD affinity matrix
        # M = torch.zeros((n+m, n+m))
        # for i in range(n):
        #     for j in range(m):
        #         M[i, j+n] = M[j+n, i] = K_xy[i, j] - mu_xx/n - mu_yy/m + mu_xy

#     else:
#         raise ValueError("Unsupported kernel type.")
#     # mmd = torch.zeros((n+m, n+m))
#     # mmd = torch.zeros((x.shape[0], y.shape[0]))
#     # for i in range(x.shape[0]):
#     #     for j in range(y.shape[0]):
#     #         mmd[i, j] = x_kernel[i] + y_kernel[j] - 2 * torch.matmul(x[i], y[j])
#     return mmd
# # mmd_affinity = mmd_affinity_matrix(image_embeddings, text_embeddings, kernel_type='gaussian', sigma=1.0)
# # print(mmd_affinity.shape)

In [9]:
# def linear_kernel(x, y):
#     return torch.dot(x, y)
#
# def gaussian_kernel(x, y, sigma=1.0):
#     norm = torch.norm(x-y)
#     return torch.exp(-norm**2 / (2 * (sigma ** 2)))
#
# def polynomial_kernel(x, y, degree=2):
#     return (torch.dot(x, y) + 1) ** degree
#
# def mmd_affinity_matrix(X, Y, kernel_type='linear', degree=2, sigma=None):
#     # Calculate the kernel matrix between X and Y
#     kernel=kernel_type
#     if kernel == 'linear':
#         KXX = torch.zeros((X.shape[0], X.shape[0]))
#         KYY = torch.zeros((Y.shape[0], Y.shape[0]))
#         KXY = torch.zeros((X.shape[0], Y.shape[0]))
#
#         for i in range(X.shape[0]):
#             for j in range(X.shape[0]):
#                 KXX[i,j] = linear_kernel(X[i], X[j])
#
#         for i in range(Y.shape[0]):
#             for j in range(Y.shape[0]):
#                 KYY[i,j] = linear_kernel(Y[i], Y[j])
#
#         for i in range(X.shape[0]):
#             for j in range(Y.shape[0]):
#                 KXY[i,j] = linear_kernel(X[i], Y[j])
#     elif kernel == 'gaussian':
#         KXX = torch.zeros((X.shape[0], X.shape[0]))
#         KYY = torch.zeros((Y.shape[0], Y.shape[0]))
#         KXY = torch.zeros((X.shape[0], Y.shape[0]))
#
#         for i in range(X.shape[0]):
#             for j in range(X.shape[0]):
#                 KXX[i,j] = gaussian_kernel(X[i], X[j], sigma=sigma)
#
#         for i in range(Y.shape[0]):
#             for j in range(Y.shape[0]):
#                 KYY[i,j] = gaussian_kernel(Y[i], Y[j], sigma=sigma)
#
#         for i in range(X.shape[0]):
#             for j in range(Y.shape[0]):
#                 KXY[i,j] = gaussian_kernel(X[i], Y[j], sigma=sigma)
#     elif kernel == 'polynomial':
#         KXX = torch.zeros((X.shape[0], X.shape[0]))
#         KYY = torch.zeros((Y.shape[0], Y.shape[0]))
#         KXY = torch.zeros((X.shape[0], Y.shape[0]))
#
#         for i in range(X.shape[0]):
#             for j in range(X.shape[0]):
#                 KXX[i,j] = polynomial_kernel(X[i], X[j], degree=degree)
#
#         for i in range(Y.shape[0]):
#             for j in range(Y.shape[0]):
#                 KYY[i,j] = polynomial_kernel(Y[i], Y[j], degree=degree)
#
#         for i in range(X.shape[0]):
#             for j in range(Y.shape[0]):
#                 KXY[i,j] = polynomial_kernel(X[i], Y[j], degree=degree)
#
#     # Calculate the MMD kernel matrix
#     mmd_matrix = torch.zeros((X.shape[0], Y.shape[0]))
#
#     if sigma is None:
#         sigma = torch.median(KXX) + torch.median(KYY)
#
#     for i in range(X.shape[0]):
#         for j in range(Y.shape[0]):
#             mmd_matrix[i,j] = torch.exp(-(torch.norm(KXX[i,:] - KXY[i,:])**2 + torch.norm(KYY[j,:] - KXY)))
#     return mmd_matrix


In [11]:
utils.make_output_dir(output_dir)
inputs = list(enumerate(sorted(Path(features_dir).iterdir())))
for inp in tqdm(inputs):
    index, features_file = inp
    #     print(index, features_file)
    # Load
    data_dict = torch.load(features_file, map_location='cpu')
    #     print(data_dict.keys())   #['k', 'indices', 'file', 'id', 'model_name', 'patch_size', 'shape']
    #     print("shape=", data_dict['shape'], "k shape", data_dict['k'].shape, "patch_size=", data_dict['patch_size'])
    image_id = data_dict['file'][:-4]
    #     print(image_id)
    # Load
    output_file = str(Path(output_dir) / f'{image_id}.pth')
    if Path(output_file).is_file():
        print(f'Skipping existing file {str(output_file)}')
        # break
        # return  # skip because already generated

    # Load affinity matrix
    feats = data_dict[which_features].squeeze().cuda()
    # print("Without normalizing, Features Shape is", feats.shape)
    if normalize:
        feats = F.normalize(feats, p=2, dim=-1)
    # print("After normalization, Features Shape", feats.shape)
    #     print("which_matrix=", which_matrix)
    # Eigenvectors of affinity matrix
    if which_matrix == 'affinity_torch':
        # W = feats @ feats.T
        W = mmd_affinity_matrix(feats, feats, kernel_type=kernel_type, sigma=sigma)
        # print("W_feat shape=", W_feat.shape)
        if threshold_at_zero:
            W = (W * (W > 0))
            # print("W shape=", W.shape)
        eigenvalues, eigenvectors = torch.eig(W, eigenvectors=True)
        eigenvalues = eigenvalues.cpu()
        eigenvectors = eigenvectors.cpu()
        print("which matrix=", which_matrix, "eigenvalues shape", eigenvalues.shape, "eigenvectors shape",
              eigenvectors.shape)


    # Eigenvectors of affinity matrix with scipy
    elif which_matrix == 'affinity_svd':
        USV = torch.linalg.svd(feats, full_matrices=False)
        eigenvectors = USV[0][:, :K].T.to('cpu', non_blocking=True)
        eigenvalues = USV[1][:K].to('cpu', non_blocking=True)
        print("which matrix=", which_matrix, "eigenvalues shape", eigenvalues.shape, "eigenvectors shape",
              eigenvectors.shape)

    # Eigenvectors of affinity matrix with scipy
    elif which_matrix == 'affinity':
        #         print("Without normalizing, Features Shape is",feats.shape)
        # W = (feats @ feats.T)
        W = mmd_affinity_matrix(feats, feats, kernel_type=kernel_type, sigma=sigma)
        # print("W shape=", W.shape)
        if threshold_at_zero:
            W = (W * (W > 0))
        W = W.cpu().numpy()
        # print("W shape=", W.shape)
        eigenvalues, eigenvectors = eigsh(W, which='LM', k=K)
        eigenvectors = torch.flip(torch.from_numpy(eigenvectors), dims=(-1,)).T
        print("which matrix=", which_matrix, "eigenvalues shape", eigenvalues.shape, "eigenvectors shape",
              eigenvectors.shape)

    # Eigenvectors of matting laplacian matrix
    elif which_matrix in ['matting_laplacian', 'laplacian']:

        # Get sizes
        B, C, H, W, P, H_patch, W_patch, H_pad, W_pad = utils.get_image_sizes(data_dict)
        if image_downsample_factor is None:
            image_downsample_factor = P
        H_pad_lr, W_pad_lr = H_pad // image_downsample_factor, W_pad // image_downsample_factor

        # Upscale features to match the resolution
        if (H_patch, W_patch) != (H_pad_lr, W_pad_lr):
            feats = F.interpolate(
                feats.T.reshape(1, -1, H_patch, W_patch),
                size=(H_pad_lr, W_pad_lr), mode='bilinear', align_corners=False
            ).reshape(-1, H_pad_lr * W_pad_lr).T

        ### Feature affinities
        #         print("Without normalizing, Features Shape is",feats.shape)
        # W_feat = (feats @ feats.T)
        W_feat = mmd_affinity_matrix(feats, feats, kernel_type=kernel_type, sigma=sigma)
        print("kernel type=", kernel_type, "sigma=", sigma)
        if threshold_at_zero:
            W_feat = (W_feat * (W_feat > 0))
        W_feat = W_feat / W_feat.max()  # NOTE: If features are normalized, this naturally does nothing
        W_feat = W_feat.cpu().numpy()
        # print("W_feat shape=", W_feat.shape)

        ### Color affinities
        # If we are fusing with color affinites, then load the image and compute
        if image_color_lambda > 0:

            # Load image
            image_file = str(Path(images_root) / f'{image_id}.jpg')
            image_lr = Image.open(image_file).resize((W_pad_lr, H_pad_lr), Image.BILINEAR)
            image_lr = np.array(image_lr) / 255.

            # Color affinities (of type scipy.sparse.csr_matrix)
            if which_color_matrix == 'knn':
                W_lr = utils.knn_affinity(image_lr / 255)
            elif which_color_matrix == 'rw':
                W_lr = utils.rw_affinity(image_lr / 255)

            # Convert to dense numpy array
            W_color = np.array(W_lr.todense().astype(np.float32))
            # print("W_color shape", W_color.shape)

        else:

            # No color affinity
            W_color = 0

        # Combine
        W_comb = W_feat + W_color * image_color_lambda  # combination
        D_comb = np.array(utils.get_diagonal(W_comb).todense())  # is dense or sparse faster? not sure, should check
        #         print("W_comb shape= ", W_comb.shape, "D_comb shape",  D_comb.shape)

        # Extract eigenvectors
        if lapnorm:
            try:
                eigenvalues, eigenvectors = eigsh(D_comb - W_comb, k=K, sigma=0, which='LM', M=D_comb)
            except:
                eigenvalues, eigenvectors = eigsh(D_comb - W_comb, k=K, which='SM', M=D_comb)
        else:
            try:
                eigenvalues, eigenvectors = eigsh(D_comb - W_comb, k=K, sigma=0, which='LM')
            except:
                eigenvalues, eigenvectors = eigsh(D_comb - W_comb, k=K, which='SM')
        eigenvalues, eigenvectors = torch.from_numpy(eigenvalues), torch.from_numpy(eigenvectors.T).float()
    print("eigenvalues shape", eigenvalues.shape, "eigenvectors shape", eigenvectors.shape)
    # Sign ambiguity
    for k in range(eigenvectors.shape[0]):
        if 0.5 < torch.mean((eigenvectors[k] > 0).float()).item() < 1.0:  # reverse segment
            eigenvectors[k] = 0 - eigenvectors[k]

    # Save dict
    output_dict = {'eigenvalues': eigenvalues, 'eigenvectors': eigenvectors}
    torch.save(output_dict, output_file)

  0%|          | 0/1000 [00:00<?, ?it/s]

kernel type= polynomial sigma= 1.0


  0%|          | 1/1000 [00:22<6:22:35, 22.98s/it]

eigenvalues shape torch.Size([5]) eigenvectors shape torch.Size([5, 800])
kernel type= polynomial sigma= 1.0


  0%|          | 2/1000 [00:40<5:32:03, 19.96s/it]

...

eigenvalues shape torch.Size([5]) eigenvectors shape torch.Size([5, 900])
kernel type= polynomial sigma= 1.0


100%|█████████▉| 999/1000 [9:42:03<00:19, 19.47s/it]

eigenvalues shape torch.Size([5]) eigenvectors shape torch.Size([5, 750])
kernel type= polynomial sigma= 1.0


100%|██████████| 1000/1000 [9:42:20<00:00, 34.94s/it]

eigenvalues shape torch.Size([5]) eigenvectors shape torch.Size([5, 800])
